# Assignment 5

Please answer each question to the corresponding question cell below. Your final code must have the code as well as the output of your code. You can use Saint Peter's [Azure Databricks](https://adb-7130196131129306.6.azuredatabricks.net/?o=7130196131129306#) to do this assignment.

## Questions

### Q1

Using only RDD APIs, aggregate all the ages for each name, group by name, and then average the ages for the data below.

In [0]:
data = [("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)]

In [0]:
rdd=spark.sparkContext.parallelize(data)
rdd.groupByKey().mapValues(lambda x: sum(x) / len(x)).collect()

Out[74]: [('Brooke', 22.5), ('Denny', 31.0), ('TD', 35.0), ('Jules', 30.0)]

### Q2

Do Q1 using DataFrame API. Explain the difference.

In [0]:
rdd=spark.sparkContext.parallelize(data)
columns = ["name","age"]
df = rdd.toDF(columns)
df.printSchema()
df.groupBy('name').mean('age').show()

root
-- name: string (nullable = true)
-- age: long (nullable = true)

+------+--------+
 name|avg(age)|
+------+--------+
Brooke| 22.5|
 Denny| 31.0|
 Jules| 30.0|
 TD| 35.0|
+------+--------+

For this one we had to define the columns names and upon aggregation using pyspark dataframe API it will give us a dataframe.

### Q3

Save your results from Q2 as a temporary view. Using this view, do Q1 using SQL API. See [PySpark API reference](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.createOrReplaceTempView.html?highlight=create#pyspark-sql-dataframe-createorreplacetempview) for creating temporary views from a DataFrame.

In [0]:
df.createOrReplaceTempView("data_table")
spark.sql("SELECT name, AVG(age) FROM data_table GROUP BY name").show()

+------+--------+
 name|avg(age)|
+------+--------+
Brooke| 22.5|
 Denny| 31.0|
 Jules| 30.0|
 TD| 35.0|
+------+--------+

### Q4

Using the following data structure below, create a dataframe by adding data types and column names. Colum names and the corresponding data types are:

| Column | Data Type
-- | ---
|`Id` | INT
|`First` | STRING 
|`Last` | STRING
|`Url` | STRING
|`Published` | STRING
|`Hits` | INT
|`Campaigns` | ARRAY[STRING]
  
Print the schema of your DataFrame. Explain what is the main advantage of adding data types while creating DataFrames.

In [0]:
from pyspark.sql.types import *

user_defined_schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("First", StringType(), True),
    StructField("Last", StringType(), True),
    StructField("Url", StringType(), True),
    StructField("Published", StringType(), True),
    StructField("Hits", IntegerType(), True),
    StructField("Campaigns", ArrayType(StringType()), True)
])

In [0]:
data = [   
    [1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
    [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
    [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web", "twitter", "FB", "LinkedIn"]],
    [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
    [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]], 
    [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]],
]

In [0]:
rdd1 = spark.sparkContext.parallelize(data)
dfFromRDD1 = rdd1.toDF(user_defined_schema)
dfFromRDD1.printSchema()

root
-- ID: integer (nullable = true)
-- First: string (nullable = true)
-- Last: string (nullable = true)
-- Url: string (nullable = true)
-- Published: string (nullable = true)
-- Hits: integer (nullable = true)
-- Campaigns: array (nullable = true)
 |-- element: string (containsNull = true)

In [0]:
dfFromRDD1.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
 ID| First| Last| Url|Published| Hits| Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

The advantages of adding data types while creating a dataframe:
1. You relieve Spark from the onus of inferring data types.
2. You prevent Spark from creating a separate job just to read a large portion of your file to ascertain the schema, which for a large data file can be expensive and time-consuming.
3. You can detect errors early if data doesn’t match the schema.

### Q5

Add a new column to the DataFrame created in Q4 with the following specs:

- Column name is `Big Hitters`
- Values will be `True` or `False`. `True` if the column `Hits` bigger than 10000, else `False`

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
dfFromRDD1 = dfFromRDD1.withColumn("Big Hitters", F.when(col('Hits')>10000, True).otherwise(False))
dfFromRDD1.show()

+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
 ID| First| Last| Url|Published| Hits| Campaigns|Big Hitters|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]| false|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]| false|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...| false|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]| true|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...| true|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]| true|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+

## Notes

After done with the assignment, please upload your work to the blackboard. Please make sure you upload:

- The outputs of the code in the notebook
- The pdf version of the notebook

Best of luck!